## Prediction Analysis and XAI(Explainable AI)

#### Note::: This file is still under development and not really organised

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [58]:
df_artists = pd.read_csv('../dataset/artists.csv',sep=None, index_col=0, engine='python', encoding='utf-8')
df_artists.head()

,name,gender,birth_date,birth_place,nationality,description,active_start,active_end,province,region,country,latitude,longitude
id_author,,,,,,,,,,,,,
ART82291002,99 posse,M,NaN,NaN,NaN,gruppo musicale italiano,1991-01-01,NaN,NaN,NaN,NaN,NaN,NaN
ART53496045,achille lauro,M,1990-07-11,Verona,Italia,cantautore e rapper italiano (1990-),2012-01-01,NaN,Verona,Veneto,Italia,45.442498,10.985738
ART18853907,alfa,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ART64265460,anna pepe,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ART75741740,articolo 31,M,NaN,NaN,NaN,gruppo musicale hip hop italiano,1990-01-01,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_artists

,name,region
id_author,,
ART53496045,achille lauro,Veneto
ART24123617,babaman,Lombardia
ART56320683,bassi maestro,Lombardia
ART27304446,caparezza,Puglia
ART70825116,capo plaza,Campania
...,...,...
ART57242110,tony effe,Lazio
ART98118784,tormento,Calabria
ART15560128,vacca,Sardegna


In [49]:
df_artists.dtypes

id_author        object
name             object
gender           object
birth_date       object
birth_place      object
nationality      object
description      object
active_start     object
active_end      float64
province         object
region           object
country          object
latitude        float64
longitude       float64
dtype: object

#### We first have to clean the artists whose regions are null

In [55]:
df_artists.isna().sum()


name              0
gender            0
birth_date       31
birth_place      32
nationality      33
description      18
active_start     54
active_end      104
province         34
region           36
country          34
latitude         32
longitude        32
dtype: int64

In [72]:
df_artists

,name,region
id_author,,
ART53496045,achille lauro,Veneto
ART24123617,babaman,Lombardia
ART56320683,bassi maestro,Lombardia
ART27304446,caparezza,Puglia
ART70825116,capo plaza,Campania
...,...,...
ART57242110,tony effe,Lazio
ART98118784,tormento,Calabria
ART15560128,vacca,Sardegna


In [117]:
# ##### I have to remove some column because i do not need them for the prediction

# unneeded_columns = ['gender', 'birth_date', 'birth_place', 'nationality', 'description', 'description', 'active_start', 'active_end', 'country', 'latitude', 'longitude', 'province']
# df_artists = df_artists.drop(columns=unneeded_columns)

In [118]:
df_artists

,name,region
id_author,,
ART53496045,achille lauro,Veneto
ART24123617,babaman,Lombardia
ART56320683,bassi maestro,Lombardia
ART27304446,caparezza,Puglia
ART70825116,capo plaza,Campania
...,...,...
ART57242110,tony effe,Lazio
ART98118784,tormento,Calabria
ART15560128,vacca,Sardegna


In [126]:
df_artists.drop(columns='name', inplace=True)

/tmp/ipykernel_35967/475785859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists.drop(columns='name', inplace=True)


In [127]:
df_artists.isnull().sum()

region    0
dtype: int64

In [124]:
df_artists.dropna(inplace=True)


/tmp/ipykernel_35967/2034816017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists.dropna(inplace=True)


In [128]:
df_artists.head()

,region
id_author,
ART53496045,Veneto
ART24123617,Lombardia
ART56320683,Lombardia
ART27304446,Puglia
ART70825116,Campania


### Now we can create macro-zones column

In [133]:
from_regions_to_macro_zones = {
    # North
    'Lombardia': 'North',
    'Piemonte': 'North', 
    'Veneto': 'North', 
    'Liguria': 'North',
    
    # Campania
    'Campania': 'Campania',
    
    #Lazio
    'Lazio': 'Lazio',
    
    #Sardegna
    'Sardegna': 'Sardegna',
    
    #Center
    'Toscana':'Center',
    'Emilia-Romagna':'Center',
    'Marche':'Center',
    
    #South
    'Puglia': 'South', 
    'Sicilia': 'South', 
    'Calabria': 'South'  
}


In [135]:
### We are now comparing the regions with the marcozones while also creating the marcozones column
df_artists['marco-zones'] = df_artists['region'].map(from_regions_to_macro_zones)

/tmp/ipykernel_35967/3295486517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists['marco-zones'] = df_artists['region'].map(from_regions_to_macro_zones)


In [149]:
df_artists.head(10)

,region,marco-zones
id_author,,
ART53496045,Veneto,North
ART24123617,Lombardia,North
ART56320683,Lombardia,North
ART27304446,Puglia,South
ART70825116,Campania,Campania
ART67409252,Liguria,North
ART71969350,Campania,Campania
ART78209349,Campania,Campania
ART46851094,Liguria,North


## We then explore the tracks dataset

In [79]:
df_tracks = pd.read_csv('../dataset/tracks.csv',sep=None, engine='python', encoding='utf-8')
df_tracks.head()

,id,id_artist,name_artist,full_title,title,featured_artists,primary_artist,language,album,stats_pageviews,...,album_type,disc_number,track_number,duration_ms,explicit,popularity,album_image,id_album,lyrics,modified_popularity
0,TR934808,ART04205421,Rosa Chemical,​polka 2 :-/ by Rosa Chemical (Ft. Ernia & Guè),​polka 2 :-/,"Ernia, Guè",Rosa Chemical,pl,FOREVER AND EVER,196033.0,...,album,1.0,3.0,207761.0,True,46.0,https://i.scdn.co/image/ab67616d0000b2736d5e14...,ALB115557,"Oplà, ah\nBdope, chiama due b—\n\nMi candidere...",False
1,TR760029,ART04205421,Rosa Chemical,POLKA by Rosa Chemical (Ft. Thelonious B.),POLKA,Thelonious B.,Rosa Chemical,en,FOREVER AND EVER,164450.0,...,album,1.0,3.0,207761.0,True,46.0,https://i.scdn.co/image/ab67616d0000b2736d5e14...,ALB115557,"Greg Willen, non dormire\n(Brr-poh)\n\nT-T-Tro...",False
2,TR916821,ART04205421,Rosa Chemical,​britney ;-) by Rosa Chemical (Ft. MamboLosco ...,​britney ;-),"MamboLosco, RADICAL",Rosa Chemical,en,FOREVER AND EVER,58313.0,...,album,1.0,1.0,193544.0,True,39.0,https://i.scdn.co/image/ab67616d0000b2736d5e14...,ALB115557,"Mothz\nYeah, yeah, yeah-yeah\nBdope, chiama du...",False
3,TR480968,ART04205421,Rosa Chemical,CEO by Rosa Chemical (Ft. Taxi B),CEO,Taxi B,Rosa Chemical,it,OKAY OKAY !! - EP,39890.0,...,single,1.0,2.0,169000.0,True,47.0,https://i.scdn.co/image/ab67616d0000b27367c03d...,ALB730959,Designer sui vestiti penso di essere un outlet...,False
4,TR585039,ART04205421,Rosa Chemical,LONDRA by Rosa Chemical (Ft. Rkomi),LONDRA,Rkomi,Rosa Chemical,en,FOREVER AND EVER,35432.0,...,album,1.0,8.0,194779.0,True,41.0,https://i.scdn.co/image/ab67616d0000b273fcdb60...,ALB436151,"Bdope (Yeah)\n\nVuole solo me, non fare la gel...",False


In [90]:
df_tracks.shape

(11166, 45)

In [93]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11166 entries, 0 to 11165
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    11166 non-null  object 
 1   id_artist             11166 non-null  object 
 2   name_artist           11166 non-null  object 
 3   full_title            11166 non-null  object 
 4   title                 11166 non-null  object 
 5   featured_artists      3517 non-null   object 
 6   primary_artist        11166 non-null  object 
 7   language              11061 non-null  object 
 8   album                 9652 non-null   object 
 9   stats_pageviews       4642 non-null   float64
 10  swear_IT              11166 non-null  int64  
 11  swear_EN              11166 non-null  int64  
 12  swear_IT_words        11166 non-null  object 
 13  swear_EN_words        11166 non-null  object 
 14  year                  10766 non-null  object 
 15  month              

In [98]:
df_tracks.columns

Index(['id', 'id_artist', 'name_artist', 'full_title', 'title',
       'featured_artists', 'primary_artist', 'language', 'album',
       'stats_pageviews', 'swear_IT', 'swear_EN', 'swear_IT_words',
       'swear_EN_words', 'year', 'month', 'day', 'n_sentences', 'n_tokens',
       'tokens_per_sent', 'char_per_tok', 'lexical_density',
       'avg_token_per_clause', 'bpm', 'centroid', 'rolloff', 'flux', 'rms',
       'zcr', 'flatness', 'spectral_complexity', 'pitch', 'loudness',
       'album_name', 'album_release_date', 'album_type', 'disc_number',
       'track_number', 'duration_ms', 'explicit', 'popularity', 'album_image',
       'id_album', 'lyrics', 'modified_popularity'],
      dtype='object')

#### We do not need these columns because they wont give clue to our model for prediction
#### If the model is not performing well, then we shall change the clues and see which one would perform well and keep them.

In [106]:
###### Let's remove columns that are not necessary here
columns_not_necessary_tracks = ['id', 'full_title', 'featured_artists', 'primary_artist', 'language', 'album', 'stats_pageviews',
                                'month', 'day', 'album_name', 'album_release_date', 'album_type', 'disc_number', 'track_number', 'album_image',
                                'id_album', 'modified_popularity', 'popularity', 'title', 'swear_IT_words', 'swear_EN_words', 'lyrics'
                                ]
df_tracks=df_tracks.drop(columns=columns_not_necessary_tracks)

In [107]:
df_tracks.head()

,id_artist,name_artist,swear_IT,swear_EN,year,n_sentences,n_tokens,tokens_per_sent,char_per_tok,lexical_density,...,rolloff,flux,rms,zcr,flatness,spectral_complexity,pitch,loudness,duration_ms,explicit
0,ART04205421,Rosa Chemical,13,6,2021.0,102.0,911.0,8.931373,4.170455,0.575284,...,2895.7767,1.4499,0.1786,0.1046,0.8202,25.7148,2311.1779,17.8675,207761.0,True
1,ART04205421,Rosa Chemical,9,12,2020.0,56.0,675.0,12.053571,4.280851,0.648936,...,3378.4605,1.3533,0.2020,0.1175,0.6739,29.1089,1892.1924,21.4595,207761.0,True
2,ART04205421,Rosa Chemical,16,12,2021.0,88.0,758.0,8.613636,4.075251,0.556856,...,2037.4847,1.3822,0.2552,0.0800,0.7842,26.9762,2484.3503,29.4590,193544.0,True
3,ART04205421,Rosa Chemical,8,3,2019.0,37.0,382.0,10.324324,4.023881,0.534328,...,1156.3781,1.5499,0.1971,0.0436,0.8764,14.2956,2984.6109,20.5067,169000.0,True
4,ART04205421,Rosa Chemical,1,0,2020.0,48.0,429.0,8.937500,3.922857,0.491429,...,1693.4542,1.0886,0.2369,0.0695,0.8571,36.6951,1572.0499,25.3407,194779.0,True


In [108]:
df_tracks.isna().sum()

id_artist                 0
name_artist               0
swear_IT                  0
swear_EN                  0
year                    400
n_sentences              76
n_tokens                 76
tokens_per_sent          76
char_per_tok             76
lexical_density          76
avg_token_per_clause     76
bpm                      64
centroid                 64
rolloff                  64
flux                     64
rms                      64
zcr                      64
flatness                 64
spectral_complexity      64
pitch                    64
loudness                 64
duration_ms              78
explicit                 78
dtype: int64

In [109]:
df_tracks.dropna(inplace=True)

In [112]:
df_tracks.isna().sum()

id_artist               0
name_artist             0
swear_IT                0
swear_EN                0
year                    0
n_sentences             0
n_tokens                0
tokens_per_sent         0
char_per_tok            0
lexical_density         0
avg_token_per_clause    0
bpm                     0
centroid                0
rolloff                 0
flux                    0
rms                     0
zcr                     0
flatness                0
spectral_complexity     0
pitch                   0
loudness                0
duration_ms             0
explicit                0
dtype: int64

In [113]:
df_tracks.shape

(10634, 23)

In [114]:
df_tracks.head()

,id_artist,name_artist,swear_IT,swear_EN,year,n_sentences,n_tokens,tokens_per_sent,char_per_tok,lexical_density,...,rolloff,flux,rms,zcr,flatness,spectral_complexity,pitch,loudness,duration_ms,explicit
0,ART04205421,Rosa Chemical,13,6,2021.0,102.0,911.0,8.931373,4.170455,0.575284,...,2895.7767,1.4499,0.1786,0.1046,0.8202,25.7148,2311.1779,17.8675,207761.0,True
1,ART04205421,Rosa Chemical,9,12,2020.0,56.0,675.0,12.053571,4.280851,0.648936,...,3378.4605,1.3533,0.2020,0.1175,0.6739,29.1089,1892.1924,21.4595,207761.0,True
2,ART04205421,Rosa Chemical,16,12,2021.0,88.0,758.0,8.613636,4.075251,0.556856,...,2037.4847,1.3822,0.2552,0.0800,0.7842,26.9762,2484.3503,29.4590,193544.0,True
3,ART04205421,Rosa Chemical,8,3,2019.0,37.0,382.0,10.324324,4.023881,0.534328,...,1156.3781,1.5499,0.1971,0.0436,0.8764,14.2956,2984.6109,20.5067,169000.0,True
4,ART04205421,Rosa Chemical,1,0,2020.0,48.0,429.0,8.937500,3.922857,0.491429,...,1693.4542,1.0886,0.2369,0.0695,0.8571,36.6951,1572.0499,25.3407,194779.0,True


In [150]:
df_artists.head()

,region,marco-zones
id_author,,
ART53496045,Veneto,North
ART24123617,Lombardia,North
ART56320683,Lombardia,North
ART27304446,Puglia,South
ART70825116,Campania,Campania


### Then we merge the dataset to get only one final dataset for modeling

In [151]:
final_model_df = pd.merge(
    df_tracks,
    df_artists,
    left_on='id_artist',
    right_on='id_author'
)

In [152]:
final_model_df.head()

,id_artist,name_artist,swear_IT,swear_EN,year,n_sentences,n_tokens,tokens_per_sent,char_per_tok,lexical_density,...,rms,zcr,flatness,spectral_complexity,pitch,loudness,duration_ms,explicit,region,marco-zones
0,ART04205421,Rosa Chemical,13,6,2021.0,102.0,911.0,8.931373,4.170455,0.575284,...,0.1786,0.1046,0.8202,25.7148,2311.1779,17.8675,207761.0,True,Piemonte,North
1,ART04205421,Rosa Chemical,9,12,2020.0,56.0,675.0,12.053571,4.280851,0.648936,...,0.2020,0.1175,0.6739,29.1089,1892.1924,21.4595,207761.0,True,Piemonte,North
2,ART04205421,Rosa Chemical,16,12,2021.0,88.0,758.0,8.613636,4.075251,0.556856,...,0.2552,0.0800,0.7842,26.9762,2484.3503,29.4590,193544.0,True,Piemonte,North
3,ART04205421,Rosa Chemical,8,3,2019.0,37.0,382.0,10.324324,4.023881,0.534328,...,0.1971,0.0436,0.8764,14.2956,2984.6109,20.5067,169000.0,True,Piemonte,North
4,ART04205421,Rosa Chemical,1,0,2020.0,48.0,429.0,8.937500,3.922857,0.491429,...,0.2369,0.0695,0.8571,36.6951,1572.0499,25.3407,194779.0,True,Piemonte,North


In [160]:
## For our final dataset, we have to drop irrelevant column that wont add value such as 
# Id_artist, name_artist, and their respective region since we have their macro-zones
final_model_df.drop(columns=["id_artist", "name_artist", "region"], inplace=True)

In [164]:
final_model_df.tail()

,swear_IT,swear_EN,year,n_sentences,n_tokens,tokens_per_sent,char_per_tok,lexical_density,avg_token_per_clause,bpm,...,flux,rms,zcr,flatness,spectral_complexity,pitch,loudness,duration_ms,explicit,marco-zones
7639,2,0,1922,82.0,536.0,6.536585,4.099426,0.598470,4.123077,108.00,...,1.2193,0.3131,0.0498,0.9283,33.0093,2913.4473,36.2754,292000.0,False,South
7640,0,0,8i0piY,22.0,122.0,5.545455,4.317073,0.585366,10.166667,112.46,...,0.9720,0.1183,0.0532,0.8513,26.9888,1382.8082,9.9628,60230.0,False,South
7641,3,0,1938,101.0,737.0,7.297030,4.267496,0.528771,8.375000,103.88,...,1.3565,0.2599,0.0504,0.9247,23.9077,2725.4171,29.0896,236000.0,False,South
7642,1,0,2015.0,76.0,602.0,7.921053,4.098004,0.500907,5.016667,172.27,...,1.1842,0.2960,0.0707,0.9159,41.3077,2537.0837,33.5024,248330.0,True,South
7643,0,1,2005.0,32.0,304.0,9.500000,3.824373,0.465950,7.238095,86.99,...,1.2661,0.2901,0.0638,0.9404,26.1900,2754.6887,32.5964,302173.0,True,South


In [162]:
final_model_df.shape

(7644, 22)

In [153]:
final_model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7644 entries, 0 to 7643
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_artist             7644 non-null   object 
 1   name_artist           7644 non-null   object 
 2   swear_IT              7644 non-null   int64  
 3   swear_EN              7644 non-null   int64  
 4   year                  7644 non-null   object 
 5   n_sentences           7644 non-null   float64
 6   n_tokens              7644 non-null   float64
 7   tokens_per_sent       7644 non-null   float64
 8   char_per_tok          7644 non-null   float64
 9   lexical_density       7644 non-null   float64
 10  avg_token_per_clause  7644 non-null   float64
 11  bpm                   7644 non-null   float64
 12  centroid              7644 non-null   float64
 13  rolloff               7644 non-null   float64
 14  flux                  7644 non-null   float64
 15  rms                  

In [158]:
final_model_df.isnull().sum()

id_artist               0
name_artist             0
swear_IT                0
swear_EN                0
year                    0
n_sentences             0
n_tokens                0
tokens_per_sent         0
char_per_tok            0
lexical_density         0
avg_token_per_clause    0
bpm                     0
centroid                0
rolloff                 0
flux                    0
rms                     0
zcr                     0
flatness                0
spectral_complexity     0
pitch                   0
loudness                0
duration_ms             0
explicit                0
region                  0
marco-zones             0
dtype: int64

In [159]:
final_model_df.columns

Index(['id_artist', 'name_artist', 'swear_IT', 'swear_EN', 'year',
       'n_sentences', 'n_tokens', 'tokens_per_sent', 'char_per_tok',
       'lexical_density', 'avg_token_per_clause', 'bpm', 'centroid', 'rolloff',
       'flux', 'rms', 'zcr', 'flatness', 'spectral_complexity', 'pitch',
       'loudness', 'duration_ms', 'explicit', 'region', 'marco-zones'],
      dtype='object')

# Model Predictions  

### Logistic Regression

hello world
